### Exercise
- Apply [Negation Marking](https://www.nltk.org/_modules/nltk/sentiment/util.html#mark_negation) to Movie Reviews Dataset
    - expects list as input
- Train SVM model
- Compare results to Naive Bayes performance

In [8]:
from nltk.sentiment.util import mark_negation
print(mark_negation("This is not cool".split()))

['This', 'is', 'not', 'cool_NEG']


In [14]:

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

new_neg = []
for rev in rev_neg:
    new_rev = []
    for sentence in rev:
        new_rev.append(mark_negation(sentence))
    new_neg.append(new_rev)

new_pos = []
for rev in rev_pos:
    new_rev = []
    for sentence in rev:
        new_rev.append(mark_negation(sentence))
    new_pos.append(new_rev)
    
new_corpus = [lol2str(d) for d in new_neg] + [lol2str(d) for d in rev_pos]
vectors = vectorizer.fit_transform(new_corpus)
ref = numpy.array([0] * len(new_neg) + [1] * len(new_pos))

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
scores_svm = []
scores_nb = []
for i, (train_index, test_index) in enumerate(skf.split(new_corpus, ref)):
    x_train, x_test = [new_corpus[indx] for indx in train_index], [new_corpus[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    svm_classifier = LinearSVC(C=100, max_iter=10000, dual=True)
    nb_classifier = MultinomialNB()
    
    clf_svm = svm_classifier.fit(train_features, y_train)
    hyp_svm = clf_svm.predict(test_features)
#     print(classification_report(y_test, hyp_svm))
    scores_svm.append(f1_score(y_test, hyp_svm, average='macro'))
    clf_nb = nb_classifier.fit(train_features, y_train)
    hyp_nb = clf_nb.predict(test_features)
#     print(classification_report(y_test, hyp_nb))
    scores_nb.append(f1_score(y_test, hyp_nb, average='macro'))
    
print('SVM:', round(sum(scores_svm)/len(scores_svm), 3))
print('NB:', round(sum(scores_nb)/len(scores_nb), 3))


SVM: 0.959
NB: 0.961


### Exercises

1. Implement classification using counts of negative and positive terms (i.e. convert scores to label at token-level and count those for a document)
2. Implement classification using counts of negative and positive sentences (i.e. score sentences and aggregate their labels)
3. Train and evaluate supervise machine learning model by first removing objective sentences.
4. Classify Movie Reviews using VADER.
5. Do a proper cross-validation evaluation

In [34]:
import numpy as np
print("WORD LEVEL")
print("="*89)
def polarity_word_level(document, analyzer):
    pos = 0
    neg = 0
    labels = ['P', 'N']
    for sentence in document:
        for w in sentence:
            value = analyzer.polarity_scores(w)
            if value["compound"] > 0:
                pos += 1 
            elif value["compound"] < 0:
                neg += 1
    return labels[np.argmax(np.asarray([pos, neg]))]

hyp_word_level =  [polarity_word_level(doc, analyzer) for doc in rev_neg] + \
                  [polarity_word_level(doc, analyzer) for doc in rev_pos]
print(classification_report(ref, hyp_word_level))    
print("="*89)

WORD LEVEL
              precision    recall  f1-score   support

           N       0.73      0.32      0.44      1000
           P       0.56      0.88      0.69      1000

    accuracy                           0.60      2000
   macro avg       0.65      0.60      0.57      2000
weighted avg       0.65      0.60      0.57      2000

SENTENCE LEVEL
              precision    recall  f1-score   support

           N       0.72      0.39      0.51      1000
           P       0.58      0.85      0.69      1000

    accuracy                           0.62      2000
   macro avg       0.65      0.62      0.60      2000
weighted avg       0.65      0.62      0.60      2000



In [45]:
print("SENTENCE LEVEL")
print("="*89)

def polarity_sentence_level(document, analyzer):
    pos = 0
    neg = 0
    labels = ['P', 'N']
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] > 0:
            pos += 1 
        elif value["compound"] < 0:
            neg += 1
    return labels[np.argmax(np.asarray([pos, neg]))]

hyp_sentence_level =  [polarity_sentence_level(doc, analyzer) for doc in rev_neg] + \
                  [polarity_sentence_level(doc, analyzer) for doc in rev_pos]
print(classification_report(ref, hyp_sentence_level))    


SENTENCE LEVEL
              precision    recall  f1-score   support

           N       0.72      0.39      0.51      1000
           P       0.58      0.85      0.69      1000

    accuracy                           0.62      2000
   macro avg       0.65      0.62      0.60      2000
weighted avg       0.65      0.62      0.60      2000



In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

def rm_objective_sentences(document, analyzer):
    new_doc = []
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] != 0:
            new_doc.append(" ".join(sentence))
    return new_doc
def polarity_doc_level(document, analyzer):
    value = analyzer.polarity_scores(document)
    if value["compound"] > 0:
        return 'P'
    elif value["compound"] <= 0: # In this way we penalize the neg class
        return 'N'
    
rev_neg_wo_objective = ["\n".join(rm_objective_sentences(doc, analyzer)) for doc in rev_neg]
rev_pos_wo_objective = ["\n".join(rm_objective_sentences(doc, analyzer)) for doc in rev_pos]
corpus_wo_objective = rev_neg_wo_objective + rev_pos_wo_objective

In [67]:
# Train and test with Stratified K Fold

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
scores_clf = []
scores_vader = []
scores_sentence = []
scores_word = []

for i, (train_index, test_index) in enumerate(skf.split(corpus_wo_objective, ref)):
    x_train, x_test = [corpus_wo_objective[indx] for indx in train_index], [corpus_wo_objective[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = TfidfVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    hyp_vader = [polarity_doc_level(doc, analyzer) for doc in x_test]
    scores_vader.append(f1_score(y_test, hyp_vader, average='macro'))
    
    hyp_word = [polarity_word_level(doc, analyzer) for doc in test_x_split]
    scores_word.append(f1_score(y_test, hyp_word, average='macro'))
    
    hyp_sentence = [polarity_sentence_level(doc, analyzer) for doc in test_x_split]
    scores_sentence.append(f1_score(y_test, hyp_sentence, average='macro'))
    
    
print('F1 classifier:', round(sum(scores_clf)/len(scores_clf), 3))
print('F1 VADER:',  round(sum(scores_vader)/len(scores_vader), 3))
print('F1 Word:',  round(sum(scores_word)/len(scores_word), 3))
print('F1 Sentence:',  round(sum(scores_sentence)/len(scores_sentence), 3))

F1 classifier: 0.847
F1 VADER: 0.612
F1 Word: 0.565
F1 Sentence: 0.599
